In [2]:
import json
from datetime import timedelta
from urllib.request import urlopen

#for storage and analysis of data-sets
import numpy as np
import pandas as pd
#import geopandas as gpd

#for visual graphs and imaging
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium

#colour coding
cnf='#393e46' #confirmed=grey
dth='#ff2e63' #death=red
rec='#21bf73' #recovered= cyan
act='#fe9801' #active= yellow

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
df= pd.read_csv('/content/complete.csv', parse_dates=['Date'])
df['Name of State / UT'] = df['Name of State / UT'].str.replace('Union Territory of ', '')
df.head()
df.columns

Index(['Date', 'Name of State / UT', 'Total Confirmed cases (Indian National)',
       'Total Confirmed cases ( Foreign National )',
       'Cured/Discharged/Migrated', 'Latitude', 'Longitude', 'Death',
       'Total Confirmed cases'],
      dtype='object')

In [4]:
df = df[['Date', 'Name of State / UT', 'Latitude', 'Longitude', 'Total Confirmed cases', 'Death', 'Cured/Discharged/Migrated']]
df.columns = ['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Deaths', 'Cured']
for i in ['Confirmed', 'Deaths', 'Cured']:
    df[i] = df[i].astype('int')
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Cured']
df['Mortality rate'] = df['Deaths']/df['Confirmed']
df['Recovery rate'] = df['Cured']/df['Confirmed']
df = df[['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Active', 'Deaths', 'Mortality rate', 'Cured', 'Recovery rate']]
df.head(len(df))

,Date,State/UT,Latitude,Longitude,Confirmed,Active,Deaths,Mortality rate,Cured,Recovery rate
0,2020-01-30,Kerala,10.8505,76.2711,1,1,0,0.000000,0,0.000000
1,2020-01-31,Kerala,10.8505,76.2711,1,1,0,0.000000,0,0.000000
2,2020-02-01,Kerala,10.8505,76.2711,2,2,0,0.000000,0,0.000000
3,2020-02-02,Kerala,10.8505,76.2711,3,3,0,0.000000,0,0.000000
4,2020-02-03,Kerala,10.8505,76.2711,3,3,0,0.000000,0,0.000000
...,...,...,...,...,...,...,...,...,...,...
608,2020-04-03,Tamil Nadu,11.1271,78.6569,309,302,1,0.003236,6,0.019417
609,2020-04-03,Telengana,18.1124,79.0193,107,103,3,0.028037,1,0.009346
610,2020-04-03,Uttar Pradesh,26.8467,80.9462,113,97,2,0.017699,14,0.123894
611,2020-04-03,Uttarakhand,30.0668,79.0193,10,8,0,0.000000,2,0.200000


In [5]:
latest = df[df['Date']==max(df['Date'])]
latest_day = max(df['Date'])
day_before = latest_day - timedelta(days = 1)
latest_day_df = df[df['Date']==latest_day].set_index('State/UT')
day_before_df = df[df['Date']==day_before].set_index('State/UT')
temp = pd.merge(left = latest_day_df, right = day_before_df, on='State/UT', suffixes=('_lat', '_bfr'), how='outer')
latest_day_df['New cases'] = temp['Confirmed_lat'] - temp['Confirmed_bfr']
latest = latest_day_df.reset_index()
latest.fillna(1, inplace=True)
print(latest_day, day_before)

2020-04-03 00:00:00 2020-04-02 00:00:00


In [6]:
temp = latest[['State/UT', 'Confirmed', 'Active', 'New cases', 'Deaths', 'Mortality rate', 'Cured', 'Recovery rate']]
temp = temp.sort_values('Confirmed', ascending=False).reset_index(drop=True)
temp.style\
    .background_gradient(cmap="Blues", subset=['Confirmed', 'Active', 'New cases'])\
    .background_gradient(cmap="Greens", subset=['Cured', 'Recovery rate'])\
    .background_gradient(cmap="Reds", subset=['Deaths', 'Mortality rate'])

,State/UT,Confirmed,Active,New cases,Deaths,Mortality rate,Cured,Recovery rate
0,Maharashtra,335,277,0.000000,16,0.047761,42,0.125373
1,Tamil Nadu,309,302,75.000000,1,0.003236,6,0.019417
2,Kerala,286,257,21.000000,2,0.006993,27,0.094406
3,Delhi,219,207,67.000000,4,0.018265,8,0.036530
4,Rajasthan,133,130,25.000000,0,0.000000,3,0.022556
5,Andhra Pradesh,132,130,46.000000,1,0.007576,1,0.007576
6,Karnataka,124,111,14.000000,3,0.024194,10,0.080645
7,Uttar Pradesh,113,97,0.000000,2,0.017699,14,0.123894
8,Telengana,107,103,11.000000,3,0.028037,1,0.009346
9,Madhya Pradesh,99,93,0.000000,6,0.060606,0,0.000000


In [7]:
m = folium.Map(location=[20.5937, 78.9629], tiles='cartodbpositron',
               min_zoom=4, max_zoom=6, zoom_start=4)

for i in range(0, len(latest)):
    if latest.iloc[i]['Confirmed']>0:
        folium.Circle(
            location=[latest.iloc[i]['Latitude'], latest.iloc[i]['Longitude']],
            color='#ff2e63', 
            fill='#ff2e63',
            tooltip =   '<li><bold>Name of State / UT : '+str(latest.iloc[i]['State/UT'])+
                        '<li><bold>Confirmed cases  : '+str(latest.iloc[i]['Confirmed']),
            radius=int(latest.iloc[i]['Confirmed'])*300).add_to(m)
m